<a href="https://colab.research.google.com/github/sethkipsangmutuba/Distributed-Computing-Application/blob/main/Note2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Note 2: Interprocess Communications – Outline**

##Seth Kipsang

## **2.1 An Archetypal IPC Program Interface**

## **2.2 Event Synchronization**

### **Synchronous Send & Synchronous Receive**

### **Asynchronous Send & Synchronous Receive**

### **Synchronous Send & Asynchronous Receive**

### **Asynchronous Send & Asynchronous Receive**

## **2.3 Timeouts and Threading**

## **2.4 Deadlocks and Timeouts**

## **2.5 Data Representation**

## **2.6 Data Encoding**

## **2.7 Text-Based Protocols**

## **2.8 Request–Response Protocols**

## **2.9 Event Diagram and Sequence Diagram**

## **2.10 Connection-Oriented vs Connectionless IPC**

## **2.11 Evolution of Paradigms for Interprocess Communications**


# **Overview**
Interprocess Communication (IPC) is the backbone of distributed computing, enabling independent processes—potentially running on different machines—to exchange data and collaborate. IPC typically operates in two modes:

- **Unicast:** One-to-one communication.  
- **Multicast:** One-to-many communication.

---

# **2.1 Archetypal IPC Program Interface**

## **Primitive Operations**

### **1. Send**
- Transmits data from a sender process to a receiver process.  
- Requires identification of the **receiver** and the **data** to be sent.

### **2. Receive**
- Accepts incoming data from a sender process.  
- Requires the **sender’s identity** and **memory space** to store the incoming data.

### **3. Connect** *(Connection-Oriented IPC)*
- Establishes a logical communication channel between two processes.

### **4. Disconnect** *(Connection-Oriented IPC)*
- Terminates and deallocates an established communication channel.

---

## **Operation Semantics**
- Each primitive operation corresponds to an **event**.  
  - `send` → triggers transmission  
  - `receive` → triggers delivery  
- Processes operate **independently**, without knowledge of each other’s internal state or execution flow.

---

## **Application Layer Context**

Primitive IPC operations form the foundation of **distributed computing paradigms**.

### **Example: HTTP Communication**

#### **Web Browser Workflow**
1. `connect`  
2. `send` request  
3. `receive` response  
4. `disconnect`

#### **Web Server Workflow**
1. Accept connection  
2. `receive` request  
3. `send` response  
4. `disconnect`

---

## **Abstraction**
IPC APIs simplify development by **hiding OS-level complexities**, such as:
- Message queues  
- Semaphores  
- Shared memory segments  

This allows developers to focus on **application logic** rather than low-level synchronization or resource management.

---

## **Key Concepts**

### **1. Paradigms**
Abstract models that define how tasks are executed in IPC systems.

### **2. Event-Driven Communication**
Each IPC operation generates a **system event** (send, receive, connect, disconnect), forming the basis of event-driven communication architectures.

### **3. System vs Application-Level IPC**
- OS provides low-level primitives.  
- Distributed applications usually rely on **higher-level abstractions** (e.g., HTTP, RPC, message brokers) built on top of these primitives.



This code simulates a minimal IPC channel, tracking connection, send, and receive events, enabling a simple browser–server request-response exchange without actual network communication.

In [ ]:
# Minimal IPC abstraction

class Channel:
    def __init__(self):
        self.events = []

    def connect(self, a, b):
        self.events.append(("connect", a, b))

    def disconnect(self, a, b):
        self.events.append(("disconnect", a, b))

    def send(self, sender, receiver, data):
        self.events.append(("send", sender, receiver, data))

    def receive(self, receiver):
        for e in self.events:
            if e[0] == "send" and e[2] == receiver:
                return ("receive", receiver, e[1], e[3])

# Example: Browser–Server HTTP-like exchange
ipc = Channel()

# browser
ipc.connect("browser", "server")
ipc.send("browser", "server", "GET /")
resp = ipc.receive("server")   # server gets request

# server replies
ipc.send("server", "browser", "200 OK")
reply = ipc.receive("browser")

ipc.disconnect("browser", "server")

print(resp)
print(reply)


('receive', 'server', 'browser', 'GET /')
('receive', 'browser', 'server', '200 OK')


The results show correct bidirectional IPC semantics: the server receives the browser’s request, and the browser receives the server’s response, validating proper send/receive event ordering in a simple distributed exchange.

# **2.2 Event Synchronization**

## **Overview**
Event synchronization ensures correct sequencing of IPC operations when independently executing processes interact. It is fundamental in distributed systems, web services, and network protocol design, where the timing and ordering of events determine correctness and reliability.

---

# **Synchronization Mechanisms**

## **1. Blocking (Synchronous) Operations**
A process **suspends execution** until the IPC operation completes.

### **Characteristics**
- Execution pauses until the event (send, receive, connect) finishes.
- Managed by OS-level IPC facilities.
- Transparent to the programmer.
- **Risk:** Potential for *indefinite blocking* if the expected event never occurs.

### **Examples**
- **Connect:** A browser waits for the server to acknowledge the TCP handshake.
- **Receive:** A process halts until a message arrives.

---

## **2. Nonblocking / Asynchronous Operations**
A process **continues execution immediately**, without waiting for the IPC operation to complete.

### **Characteristics**
- Notification occurs later when the event is fulfilled.
- Enables high concurrency and responsiveness.

### **Example**
- A web server sending data to multiple clients without waiting for acknowledgment, allowing it to handle new requests concurrently.

---

# **IPC Operation Combinations**

## **1. Synchronous Send & Synchronous Receive**
- **Both sender and receiver block.**
- Sender waits until transmission completes; receiver waits for data arrival.
- Ensures **strict sequencing** and **ordered delivery**.

### **Use Case**
- Systems where data integrity and step-by-step coordination are required.

---

## **2. Asynchronous Send & Synchronous Receive**
- **Sender does not block**, continues immediately.
- **Receiver blocks** until data becomes available.

### **Use Case**
- Sender’s workflow does not depend on receiver’s readiness (e.g., logging systems, event dispatchers).

---

## **3. Synchronous Send & Asynchronous Receive**
- **Sender blocks** until data transmission is acknowledged.
- **Receiver does not block** and may access data via:
  - Immediate delivery (already available)
  - Polling (periodic checks)
  - Event handlers or callbacks

### **Use Case**
- Receiver maintains high responsiveness; sender requires confirmation.

---

## **4. Asynchronous Send & Asynchronous Receive**
- **Neither side blocks.**
- IPC facility must buffer data and notify the receiver or support polling.

### **Use Case**
- High-performance distributed systems, multicast communications, non-time-critical telemetry.

---

# **Applications in Telecommunications**

### **Web Protocols (e.g., HTTP)**
- Browser (client) typically performs **blocking** connect, send, and receive.
- Server often uses **nonblocking send** to serve multiple clients concurrently.

Workflow example:  
`Connect → Send → Receive → Disconnect`

### **Distributed Systems**
- Ensures **ordered message delivery**, **state consistency**, and **reliability**.
- Supports unicast and multicast communication patterns.

---

# **Key Takeaways**

- The choice between **synchronous** and **asynchronous** IPC affects:
  - **Latency**
  - **Throughput**
  - **Reliability**
  - **Concurrency**

- Proper event synchronization prevents:
  - Race conditions  
  - Message loss  
  - Protocol violations  

- IPC facilities abstract low-level details (thread scheduling, buffering), enabling developers to design scalable and robust distributed applications.



This code demonstrates **synchronous** and **asynchronous** IPC operations using a queue:

- `sync_send` / `sync_receive` **block until completion**.  
- `async_send` / `async_receive` use **threads** for nonblocking, concurrent message delivery.


In [ ]:
import queue
import threading
import time

class IPC:
    def __init__(self):
        self.buf = queue.Queue()   # simulates IPC buffer

    # ---- synchronous operations ----
    def sync_send(self, msg):
        self.buf.put(msg)          # blocks if buffer full
        return "sent"

    def sync_receive(self):
        return self.buf.get()      # blocks until message exists

    # ---- asynchronous operations ----
    def async_send(self, msg):
        threading.Thread(target=self.buf.put, args=(msg,), daemon=True).start()
        return "queued"

    def async_receive(self, handler):
        def listener():
            handler(self.buf.get())
        threading.Thread(target=listener, daemon=True).start()

# Demo: async send + sync receive
ipc = IPC()

ipc.async_send("hello")          # sender continues immediately
print(ipc.sync_receive())        # receiver blocks until data arrives

# Demo: sync send + async receive (callback)
def on_msg(m): print("async received:", m)
ipc.async_receive(on_msg)
ipc.sync_send("world")
time.sleep(0.05)                 # allow callback to fire


hello
async received: world


- `"hello"` shows the receiver successfully **unblocked** after the asynchronous send.  
- `"async received: world"` confirms the **callback-based asynchronous receive** executed independently.


# **2.3 Timeouts and Threading**

Blocking operations are essential for IPC synchronization, but indefinite suspension of a process is unacceptable in distributed systems. Two mechanisms mitigate this risk:

---

## **Timeouts**
Specify a maximum duration for a blocking operation. If the operation is not completed within this time, the IPC facility aborts it, freeing the process to continue. This prevents network-induced hangs, e.g., in TCP connection requests or client-server message exchanges.

---

## **Threading**
A child thread can execute the blocking operation, allowing the main thread to continue other tasks. This enables concurrent processing of multiple network operations, improving throughput and responsiveness in telecom systems, such as multi-session VoIP or signaling applications.

---

## **Key Insight**
Using timeouts and threading ensures robust and responsive IPC, maintaining reliability in environments prone to delays or failures.


This code demonstrates **IPC with timeouts and threading**:

- A **threaded receive call** prevents indefinite blocking, returning `"timeout"` if no message arrives.  
- Normal **send/receive** works when data is available within the timeout.


In [ ]:
import queue, threading

class IPC:
    def __init__(self):
        self.buf = queue.Queue()

    def send(self, msg):
        self.buf.put(msg)

    # blocking receive with timeout
    def receive(self, timeout):
        try:
            return self.buf.get(timeout=timeout)
        except queue.Empty:
            return "timeout"

# threaded blocking call
def threaded_receive(ipc, timeout, out):
    out.append(ipc.receive(timeout))

ipc = IPC()

# no message is sent → timeout occurs
result = []
t = threading.Thread(target=threaded_receive, args=(ipc, 0.1, result))
t.start(); t.join()

print(result[0])

# normal send → non-timeout receive
ipc.send("ok")
print(ipc.receive(0.1))


timeout
ok


The first output shows the blocking receive safely aborted after exceeding the timeout. The second confirms normal message delivery when data arrives within the allowed time.

# **2.4 Deadlocks and Timeouts**

Deadlocks occur in IPC when two or more processes block each other indefinitely due to incorrect sequencing of operations, such as issuing a receive instead of a send. For example, if Process 1 waits to receive data from Process 2 while Process 2 waits to receive data from Process 1, neither can proceed—resulting in a deadlock.

---

## **Key Points**

- **Deadlocks are often caused by protocol misunderstandings or programming errors.**

- **Processes remain suspended until a timeout triggers or the operating system intervenes.**

- **Timeouts are critical in distributed systems and telecom applications to prevent indefinite hangs, ensuring system reliability in services such as signaling, session control, and message routing.**


This code simulates a **deadlock scenario** in IPC:

- Both processes attempt to **receive before sending**, causing mutual blocking.  
- **Timeouts** prevent indefinite suspension, returning `"timeout"` for both receives.


In [ ]:
import queue

class IPC:
    def __init__(self): self.q = queue.Queue()

    def send(self, msg): self.q.put(msg)

    def receive(self, timeout):
        try: return self.q.get(timeout=timeout)
        except queue.Empty: return "timeout"

# Deadlock scenario: both processes try to receive first
ipc = IPC()

p1 = ipc.receive(0.1)   # waiting for P2 → no message
p2 = ipc.receive(0.1)   # waiting for P1 → no message

print(p1, p2)


timeout timeout


Both processes blocked waiting for each other’s message, so neither sent anything. The timeouts correctly prevented an infinite deadlock and allowed both operations to fail safely.

# **2.5 Data Representation**

In distributed systems, data must be represented consistently across heterogeneous hosts. At the physical layer, data is transmitted as analog signals representing binary streams. At the application layer, complex data types—such as integers, floating points, strings, arrays, and objects—require careful encoding for transmission.

---

## **Key Points**

### **Heterogeneous Hosts**
Different architectures (e.g., 32-bit big-endian vs 16-bit little-endian) require conversion of data formats to ensure correct interpretation.

---

## **Representation Schemes**

- **Sender converts data to receiver’s native format before sending.**

- **Sender transmits in its own format; receiver converts upon reception.**

- **Both sides use a standard external representation (e.g., ASN.1) for transmission.**

---

## **Data Structures**
Complex structures must be serialized (“flattened”) before transmission and reconstructed at the receiver.

---

## **Object-Oriented Considerations**
Objects include both state and behavior; transmitting objects requires object serialization to preserve data and methods across hosts.

---

This ensures accurate data exchange in networked applications, critical for telecom systems handling session data, signaling, and distributed service logic.


This code demonstrates **data representation and serialization**:

- `struct.pack` / `unpack` ensures **platform-independent integer encoding**.  
- `pickle` serializes and deserializes **complex objects** for safe transmission across heterogeneous systems.


In [ ]:
import struct, pickle

# Example: integer and string serialization
num = 1024
text = "telecom"

# ---- Standardized external representation (big-endian integer) ----
num_bytes = struct.pack(">I", num)        # 4-byte big-endian
num_recv = struct.unpack(">I", num_bytes)[0]

# ---- Object serialization for complex structures ----
obj = {"id": 1, "msg": text}
obj_bytes = pickle.dumps(obj)             # serialize
obj_recv = pickle.loads(obj_bytes)        # deserialize

print(num_recv, obj_recv)


1024 {'id': 1, 'msg': 'telecom'}


The output shows correct data representation and reconstruction: the integer 1024 was transmitted in a standard big-endian format, and the dictionary object was successfully serialized and deserialized.

# **2.6 Data Encoding**

Distributed applications require platform-independent schemes for encoding data during IPC to ensure interoperability across heterogeneous hosts. Standardized encoding schemes automate marshaling and unmarshaling, reducing programmer overhead and errors.

---

## **Key Points**

### **Character Encoding**
- **ASCII:** Maps English characters to numeric values (0–127).  
- **Unicode:** Supports a wider range of characters (0–65,535), enabling global interoperability.

### **Data Marshaling**
Converts structured data to an external network representation for transmission and back to internal representation upon receipt. Essential when hosts differ in architecture or representation.

### **Standard Encoding Schemes**
- **XDR (External Data Representation):** Low-level, supports selected data types; automated by IPC facilities.  
- **ASN.1 (Abstract Syntax Notation 1):** OSI standard; encodes complex structures, data types, and tags for network transmission.  
- **XML (Extensible Markup Language):** High-level, human-readable; supports customizable tags for application-level data exchange across heterogeneous systems.

---

These schemes are critical in telecommunications for consistent message formats, signaling data, session control, and cross-platform data exchange in distributed network services.


This code demonstrates **platform-independent data encoding and marshaling**:

- **UTF-8** preserves character data.  
- **Big-endian encoding** ensures numeric consistency.  
- **pickle** serializes and deserializes objects for cross-host IPC.


In [ ]:
import struct, pickle

# ---- Character Encoding ----
text = "telecom "
encoded_text = text.encode("utf-8")       # Unicode encoding
decoded_text = encoded_text.decode("utf-8")

# ---- XDR-like numeric encoding (big-endian) ----
num = 2025
num_bytes = struct.pack(">I", num)
num_recv = struct.unpack(">I", num_bytes)[0]

# ---- Object Marshaling (like ASN.1/XML serialization) ----
data = {"session": 1, "msg": "connect"}
data_bytes = pickle.dumps(data)           # marshal
data_recv = pickle.loads(data_bytes)      # unmarshal

print(decoded_text, num_recv, data_recv)


telecom  2025 {'session': 1, 'msg': 'connect'}


The output confirms correct platform-independent encoding:

- Unicode preserved the string with global characters,

- Big-endian numeric encoding transmitted 2025 correctly,

- Object marshaling retained the structured session data.

# **2.7 Text-Based Protocols**

Text-based protocols simplify data marshaling by using character streams (e.g., ASCII) for communication.

---

## **Key Points**

- Text-based data is **human-readable** and easily parsed by programs, facilitating debugging and monitoring.

- **Common text-based network protocols:**
  - **FTP (File Transfer Protocol):** file exchange over networks.  
  - **HTTP (Hypertext Transfer Protocol):** web content requests/responses.  
  - **SMTP (Simple Mail Transfer Protocol):** email transmission.

- Widely used in telecommunications for signaling, message exchange, and network service interactions.

- Offers ease of inspection and interoperability across heterogeneous systems, making it foundational for many distributed applications.


This code simulates a **text-based protocol exchange**:

- The client sends a **human-readable HTTP request**.  
- The server returns a **readable HTTP response**.  
- Demonstrates **ASCII-based communication** suitable for debugging and cross-platform interoperability.


In [ ]:
# Simple HTTP-like text protocol simulation
def send_request():
    return "GET /index.html HTTP/1.1\r\nHost: example.com\r\n"

def receive_response():
    return "HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n<html>...</html>"

# Client sends request and receives response
request = send_request()
response = receive_response()

print("Request:", request)
print("Response:", response)


Request: GET /index.html HTTP/1.1
Host: example.com

Response: HTTP/1.1 200 OK
Content-Type: text/html

<html>...</html>


The output illustrates a text-based protocol exchange:

- The request is human-readable and structured per HTTP standards.

- The response confirms successful transmission, demonstrating interoperability and easy inspection in distributed systems.

# **2.8 Request-Response Protocols**

**Definition:** A protocol in which one process sends a request and waits for a response from another process. This cycle may repeat iteratively until the communication task is complete.

---

## **Key Points**

- Forms the backbone of many distributed and networked applications.

- **Common examples:** FTP, HTTP, SMTP – all operate via request-response interactions.

- Critical in telecommunications systems for client-server interactions, signaling, and command-response exchanges.

- Ensures structured communication, where each request elicits a corresponding acknowledgment or data response, supporting reliable coordination between processes.


This code simulates a **request-response protocol**:

- The client iteratively sends **requests**.  
- The server returns corresponding **responses** or `"UNKNOWN"` for unrecognized requests.  
- Demonstrates **structured, reliable communication cycles**.


In [ ]:
# Simple client-server request-response simulation
def server(request):
    responses = {"HELLO": "WORLD", "PING": "PONG"}
    return responses.get(request, "UNKNOWN")

# Client sends request
requests = ["HELLO", "PING", "BYE"]
for req in requests:
    resp = server(req)
    print(f"Request: {req} -> Response: {resp}")


Request: HELLO -> Response: WORLD
Request: PING -> Response: PONG
Request: BYE -> Response: UNKNOWN


The output demonstrates a request-response protocol: each client request receives a corresponding server response. Known requests return defined replies, while unknown requests return a default acknowledgment, ensuring reliable communication.

# **2.9 Event Diagram and Sequence Diagram**

## **Event Diagrams**
- Depict the detailed sequence of events and process states (active vs. blocked) during protocol execution.  
- **Vertical lines** represent process execution over time; **solid segments = active**, **dashed segments = blocked**.  
- Useful for analyzing synchronization and blocking patterns in request-response protocols.  
- **Example:** HTTP request-response, where the browser sends a request, server responds, and processes may block/unblock accordingly.

## **Sequence Diagrams (UML)**
- Simplified, high-level abstraction of event diagrams.  
- Represent participant execution flows as **dashed lines** without differentiating blocked vs. active states.  
- Messages exchanged are shown as **directed arrows with descriptive labels**.  
- Widely used to document distributed and network protocol interactions (e.g., HTTP, FTP).

## **Telecom Engineering Context**
- Essential for designing client-server interactions, signaling protocols, and control-message exchanges.  
- Help visualize **temporal dependencies** between processes, ensuring synchronization and avoiding deadlocks in networked systems.  
- Sequence diagrams facilitate **communication protocol documentation** and **system design** for complex telecommunications networks.


This code logs **events for sequence or event diagrams**:

- Each client **request** and server **response** is recorded sequentially.  
- Produces an **ordered trace** illustrating message flow in a request-response protocol.


In [ ]:
# Event logging for sequence diagram
events = []

def client_send(req):
    events.append(f"Client sends: {req}")

def server_receive(req):
    events.append(f"Server receives: {req}")
    resp = {"HELLO": "WORLD", "PING": "PONG"}.get(req, "UNKNOWN")
    events.append(f"Server responds: {resp}")
    return resp

# Simulation
requests = ["HELLO", "PING"]
for r in requests:
    client_send(r)
    server_receive(r)

# Print events (for diagram generation)
for e in events:
    print(e)


Client sends: HELLO
Server receives: HELLO
Server responds: WORLD
Client sends: PING
Server receives: PING
Server responds: PONG


The output clearly represents a sequence of events in a request-response protocol:

- The client sends requests,

- The server receives and processes each request,

- Responses are sent back, forming an ordered trace suitable for event or sequence diagrams in distributed systems.

2.10 Connection-Oriented versus Connectionless IPC

Connection-Oriented IPC

A logical connection is established between two processes before data transfer.

Once established, the sender and receiver do not need to be repeatedly identified.

Data is exchanged sequentially over the connection.

Analogy: akin to a dedicated circuit in telecom signaling.

Connectionless IPC

Data is sent in discrete, independent packets.

Each packet must carry addressing information for the receiver.

Analogous to packet-switched networks in telecommunications, where routing occurs per message.

Telecom Engineering Context:

Connection-oriented IPC ensures reliable, ordered delivery, suitable for control-plane signaling or session-based communication.

Connectionless IPC allows flexible, low-overhead data transfer, ideal for stateless messaging or high-throughput packet transmission.

Knowledge of both paradigms is critical when designing distributed protocols or network applications using socket APIs or message-oriented middleware.

This code demonstrates **connection-oriented vs connectionless IPC**:

- **Connection-oriented:** establishes a persistent session before sending (Sent: HELLO).  
- **Connectionless:** sends independent packets with explicit addressing (Packet to Server1: HELLO).


In [ ]:
# Connection-oriented simulation
class ConnOrientedIPC:
    def __init__(self): self.connected = False
    def connect(self): self.connected = True
    def send(self, msg):
        if self.connected: return f"Sent: {msg}"
    def disconnect(self): self.connected = False

# Connectionless simulation
def connless_send(msg, receiver):
    return f"Packet to {receiver}: {msg}"

# Demo
tcp = ConnOrientedIPC()
tcp.connect()
print(tcp.send("HELLO"))
tcp.disconnect()

print(connless_send("HELLO", "Server1"))


Sent: HELLO
Packet to Server1: HELLO


The output illustrates the distinction:

- Connection-oriented IPC delivers the message over a persistent, reliable connection.

- Connectionless IPC sends an independent packet with addressing, without requiring a prior connection.

# **2.11 Evolution of IPC Paradigms**

## **Hierarchy of Abstraction in IPC**

### **1. Low-Level Data Transmission**
- Direct binary stream transfer over serial or parallel connections.  
- Suitable for network driver software or OS-level communication.  
- Not typically used in application-level distributed systems.

### **2. Socket API Paradigm**
- Provides a logical endpoint (**socket**) for each process.  
- Processes write to or read from sockets for data exchange.  
- Abstracts network addressing and packet handling.  
- Application in Java (or Winsock) programming for networked systems.

### **3. Remote Procedure Call (RPC) / Remote Method Invocation (RMI)**
- High-level abstraction allowing a process to call procedures/methods on remote processes.  
- Arguments and return values are marshaled automatically.  
- Used for distributed applications requiring seamless remote interaction, e.g., telecom service orchestration or distributed signal processing.

## **Telecom Engineering Context**
- Low-level transmission mirrors physical and link-layer protocols.  
- Socket APIs map closely to session-layer logic in networked telecom applications.  
- RPC/RMI is critical for service abstraction, enabling distributed control, monitoring, and orchestration in telecom networks and IoT systems.


This code illustrates the **evolution of IPC abstractions**:

- **Low-level:** sends raw bytes.  
- **Socket abstraction:** provides logical endpoints for communication.  
- **RPC:** enables high-level remote procedure invocation, hiding transmission details.


In [ ]:
# 1. Low-level binary transfer
def low_level_send(data_bytes):
    return f"Sent bytes: {data_bytes}"

# 2. Socket-like abstraction (simulated)
class SocketSim:
    def send(self, msg): return f"Socket sent: {msg}"
    def receive(self, msg): return f"Socket received: {msg}"

# 3. RPC-like abstraction
def remote_procedure_call(func, *args):
    return func(*args)

# Demo
print(low_level_send(b"\x01\x02"))
sock = SocketSim()
print(sock.send("Hello"))
print(remote_procedure_call(lambda x: f"Processed remotely: {x}", "Data"))


Sent bytes: b'\x01\x02'
Socket sent: Hello
Processed remotely: Data


The output demonstrates the evolution of IPC paradigms:

- Low-level transfer sends raw bytes.

- Socket abstraction provides a logical communication endpoint.

- RPC enables high-level remote procedure invocation, hiding transmission details.

# **Interprocess Communications – Summary**

## **1. Definition and Scope of IPC**
- IPC enables independent processes to communicate and collaborate in distributed computing.  
- **Unicast:** Communication between one sender and one receiver.  
- **Multicast:** Communication from one sender to a group of receivers.

---

## **2. Basic IPC API**
- **Primitive operations:** send, receive, connect, disconnect.  
- **Event synchronization:** Ensures processes execute independently while coordinating events.  
- **Blocking (synchronous) operations:** Suspend process until completion.  
- **Nonblocking (asynchronous) operations:** Allow process to continue while awaiting completion.  
- **Deadlocks:** May result from improper blocking; mitigated with timeouts or threading.

---

## **3. Data Marshaling**
Required for transmission of complex data structures. Steps include:  
- **Serialization:** Flatten data structures.  
- **External representation:** Convert values for platform-independent exchange.

**Key encoding standards:**  
- Sun XDR (External Data Representation)  
- ASN.1 (Abstract Syntax Notation One)  
- XML (Extensible Markup Language)

---

## **4. Text-Based Protocols**
- Simplest form of data marshaling involves ASCII or text streams.  
- Protocols using text encoding include **FTP, HTTP, SMTP**.

---

## **5. Request-Response Protocols**
- Processes exchange messages iteratively: **request → response → request → response**.  
- Widely used in network protocols for predictable communication patterns.

---

## **6. Visualizing IPC**
- **Event diagrams:** Show detailed timing and blocking; solid lines = active, broken lines = blocked.  
- **Sequence diagrams (UML):** Simplified representation; dashed lines for participants, messages as labeled arrows; ignores blocked vs. active states.

---

## **7. Connection Types**
- **Connection-oriented IPC:** Establish a logical connection; sender/receiver identification not needed during data exchange.  
- **Connectionless IPC:** Independent packets; each must explicitly address the receiver.

---

## **8. Levels of Abstraction in IPC**
- **Low-level:** Serial/parallel data transfer (hardware or OS-level).  
- **Mid-level:** Socket API for network programming.  
- **High-level:** Remote procedure call (RPC) or remote method invocation (RMI) for distributed applications.


This code summarizes key **IPC concepts**:

- **Primitive operations:** connect, send, receive, disconnect.  
- **Data marshaling:** object serialization and numeric encoding.  
- **Connection types:** connection-oriented vs connectionless.  
- **Request-response & event trace:** logs iterative message exchange.


In [ ]:
import queue, threading, pickle, struct

# Basic IPC primitives
class IPC:
    def __init__(self):
        self.buf = queue.Queue()
    def connect(self): return "Connected"
    def disconnect(self): return "Disconnected"
    def send(self, msg): self.buf.put(msg)
    def receive(self, timeout=None):
        try: return self.buf.get(timeout=timeout)
        except queue.Empty: return "timeout"

# Data marshaling & encoding
data = {"id":1, "msg":"hello"}
data_bytes = pickle.dumps(data)             # serialize
data_recv = pickle.loads(data_bytes)        # deserialize
num_bytes = struct.pack(">I", 1024)         # XDR-like numeric encoding
num_recv = struct.unpack(">I", num_bytes)[0]

# Connection-oriented vs connectionless
conn_msg = IPC().connect()                  # connection-oriented
connless_msg = f"Packet to Server: Hello"  # connectionless

# Request-response simulation
def server(req): return {"HELLO":"WORLD"}.get(req, "UNKNOWN")
req, resp = "HELLO", server("HELLO")

# Event trace (for sequence/event diagrams)
events = [("Client sends", req), ("Server responds", resp)]

# Output summary
print(data_recv, num_recv, conn_msg, connless_msg, events)


{'id': 1, 'msg': 'hello'} 1024 Connected Packet to Server: Hello [('Client sends', 'HELLO'), ('Server responds', 'WORLD')]


The output encapsulates the IPC summary:

- Data marshaling: Complex objects serialized/deserialized correctly.

- Numeric encoding: Platform-independent integer transfer.

- Connection-oriented vs connectionless: Demonstrates both communication styles.

- Request-response & event logging: Shows iterative message exchange and event trace for protocol analysis.